# Notebook para previsão de ações na bolsa, usando o dataset do YFinance e uso da biblioteca Pycaret

In [ ]:
#instalando as bibliotes pois o COLAB não tem como padrão
!pip install pycaret
!pip install yfinance

In [ ]:
#importando bibliotecas
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split

In [ ]:
#escolhendo uma ação
df = yf.Ticker('MRFG3.SA')
acao = df.history(period='2y')
acao.head()

In [ ]:
#excluindo alguns campos
acao = acao.drop(['Dividends','Stock Splits'], axis=1)
acao.head()

In [ ]:
#criando novos features de apoio
acao['MM07d'] = acao['Close'].rolling(window=7).mean().round(2)
acao['MM21d'] = acao['Close'].rolling(window=21).mean().round(2)
acao['MM45d'] = acao['Close'].rolling(window=45).mean().round(2)
acao.head(10)

In [ ]:
#retirando linhas com valores nulo
acao.dropna(inplace=True)
acao.head(10)

In [ ]:
# resetar os índices pois quando eliminou os nulos ficou bagunçado
acao.reset_index(drop=False,inplace=True) # para excluir colocar True no drop
acao.head(10)

In [ ]:
#Quebrando a base de dados em teste e validação
DFTreino, DFValidacao = train_test_split(acao, test_size=0.10, random_state=42)
#Avaliando as bases
print('Base Treino...: ',DFTreino.shape)
print('Base Validacao: ',DFValidacao.shape)

Base Treino...:  (409, 9)
Base Validacao:  (46, 9)


In [ ]:
#DFValidacao['Date'] = '2022-02-01 00:00:00-03:00'
#DFValidacao['Open'] = 9.28
#DFValidacao['High'] = 0
#DFValidacao['Low'] = 0
#DFValidacao['Close'] = 0
#DFValidacao['Volume'] = 0
#DFValidacao['MM07d'] = 0
#DFValidacao['MM21d'] = 0
#DFValidacao['MM45d'] = 0

In [ ]:
# importando a biblioteca pycaret
from pycaret.regression import *

# setar os dados para o modelo
setup(data = DFTreino, target='Close',session_id= 123)

In [ ]:
# 25 modelos
top3 = compare_models(n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dummy,Dummy Regressor,2.9708,13.5797,3.6546,-0.0773,0.3080,0.3054,0.0280


Processing:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
print(top3)

DummyRegressor()


In [ ]:
LR = create_model('lr',fold=10)

In [ ]:
plot_model(LR,plot='error')

In [ ]:
plot_model(LR,plot='feature')

In [ ]:
#testando com dados de treinamento
predict_model(LR)

In [ ]:
#finaliza o modelo
modelo_final = finalize_model(LR)

In [ ]:
#testando com dados de validação
previsao = predict_model(modelo_final,data=DFValidacao)
previsao.head(10)

In [ ]:
models()

In [ ]:
rna = create_model('mlp', fold=7)

In [ ]:
predict_model(rna)

In [ ]:
acao.to_csv('Ações_Mafrig.csv',header=True)